Client(Python)    RDBMS(SQLite)     DB
            (sqlite)
- Connection
- Cursor(작업) 
    1. execute*
    2. fetch
      -------->   (SQL)    <-------->
                    ?
        1. DDL (Create, Drop, Alter)
        2. DML (Insert, Select) (Delete/Update)
            Join (inner, left, right, cross)

- Insert (반드시 attribute인 column의 수와 value의 수가 일치해야한다.) -> Transaction/Commit 
    -> 1. 모든 column, 2. 특정 column, 3. column 생략
    -> execute* (params -> ?, {k(name):v(value의 값)})
    ->                          iterable 해야함. => column의 values
    -> executemany 일 떄 ------ iterable(iterable) => rows
    -> 중첩 (SQL - SELECT 괄호; group, sort, limit, ...)
- SELECT - Join
    -> 교집합(inner), left/right, cross(모든 쌍 -> cost가 크다.)
- Begin Transaction - SQLs - end
- 에러 시 try except 통해 rollback 진행.


---> 어제 진행했던 것은 ER 다이어그램이 실제 구현 시에 어떤 형식으로 구현될지 모른다.

### 오늘의 실습
-> iTunes 데이터 처리

[Artist] <- [Album] <- [Track Length Rating Count] -> [Genre]

기능 단위 작업을 하면 정렬이 될 것이다. 기능 위주.
구분을 위해 PK

[Album | id, title] <-id(album)- [Track | id, title, length, rating, album_id]
---> 해셔블할만한 key값이 있으면 좋음. PK.

In [ ]:
# 중복을 피한다. -> att

In [1]:
import sqlite3

In [2]:
con = sqlite3.connect('playlist.db')
cur = con.cursor()

In [3]:
# 4개의 테이블 생성 (가수, 앨범, 장르, 곡) <= ER(Entities)
# 가수 : PK(int), 이름(text => char/varchar)
# 앨범 : PK, 이름, 가수PK
# 장르 : PK, 이름
# 곡 : PK, 이름, 길이(time/int/real), 별점(int/real), 카운트(int), 앨범PK, 장르PK

In [8]:
# 시간 관련 타임스템프
cur.execute('SELECT CURRENT_TIMESTAMP')
cur.fetchall()

[('2023-06-28 01:49:07',)]

In [10]:
cur.executescript('''
DROP TABLE IF EXISTS artist;
CREATE TABLE artist(
    pk INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

DROP TABLE IF EXISTS album;
CREATE TABLE album(
    pk INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    fk INTEGER NOT NULL
);

DROP TABLE IF EXISTS genre;
CREATE TABLE genre(
    pk INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

DROP TABLE IF EXISTS track;
CREATE TABLE track(
    pk INTEGER PRIMARY KEY,
    name TEXT NOT NULL DEFAULT '무제',
    length INTEGER NOT NULL DEFAULT 0,
    rating INTEGER NOT NULL DEFAULT 0,
    count INTEGER NOT NULL DEFAULT 0,
    fk1 INTEGER NOT NULL,
    fk2 INTEGER NOT NULL
);

''')

In [12]:
cur.execute('INSERT INTO track(fk1, fk2) VALUES (1,1)')

In [15]:
cur.execute('SELECT * FROM track')
cur.fetchall()
# NOT NULL 제약 조건을 피해가기 위해 임의의 default 값 넣을 것임.

[(1, '무제', 0, 0, 0, 1, 1)]

In [17]:
data = [(1, '가수A'), (None, '가수B')]
cur.executemany('INSERT INTO artist VALUES(?,?)', data)

In [23]:
cur.lastrowid
# lastrowid는 한번 더 확인하자. 

2

In [44]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수A'), (2, '가수B')]

In [46]:
# 가수 입력 후 해당하는 가수가 있으면 PK, 없으면 insert 후 PK
def addArtist(name, flag = 0): # flag는 정합/부분 일치 수행용.

    # con 확인 구문 부분 -> 일반 DB에서는 connection을 항상 유지하지 않기 떄문에.
    v = None
    if flag == 0:
        v = name
    elif flag == 1:
        v = '%' + name
    elif flag == 2:
        v = name + '%'
    else:
        v = '%' + name + '%'

    cur.execute('SELECT PK FROM artist WHERE name LIKE ?', (v,))
    rst = cur.fetchone() # tuple(column...)

    if rst is None:
        cur.execute('INSERT INTO artist(name) VALUES(?)', (name,))
        rst = cur.lastrowid
    else:
        rst = rst[0] # column-PK

    return rst


In [49]:
# flag : 0 - 정확히 일치, 1 - ~로 끝나는, 2 - ~로 시작하는, 3 - 중간에
addArtist('가수', 0)

4

In [50]:
def modArtist(name, rename):
    pk = addArtist(name)

    print()
    cur.execute('UPDATE artist SET name=? WHERE pk=?', (rename, pk))

    return cur.rowcount

In [53]:
modArtist('A', '가수C')

1

In [54]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수A'), (2, '가수B'), (3, '가수C'), (4, '가수D')]

In [45]:
def delArtist(name):
    pk = addArtist(name)

    cur.execute('DELETE FROM artist WHERE pk=?', (pk,))

    return cur.rowcount

In [55]:
delArtist('가수B')

1

In [56]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수A'), (3, '가수C'), (4, '가수D')]

In [66]:
def addRow(tablename, columns, values):
    # 전제조건으로 column과 value의 개수를 맞추는 조건 주입.
    if len(columns) != len(values):
        return False
    
    # column 병합
    c = ', '.join(columns)
    q = ', '.join(['?' for v in values])

    where = list()
    for i in range(len(columns)):
        where.append('='.join([columns[i], '?'])) # name = ?
    where = ' and '.join(where)


    cur.execute('SELECT pk FROM ' + tablename + ' WHERE ' + where, values)
    rst = cur.fetchone()

    if rst == None:
        cur.execute('INSERT INTO ' + tablename + '(' + c + ')' + \
                    'VALUES(' + q + ')', values)
        rst = cur.lastrowid
    else:
        rst = rst[0]

    return rst # Auto-increment의 특성을 가지고 있는 primary-key를 통해 제대로 더해졌는지를 확인하기 위함.

In [70]:
addRow('genre', ['name'], ['클래식'])
cur.execute('SELECT * FROM genre')
cur.fetchall()

[(1, '발라드'), (2, '락'), (3, '클래식')]

In [58]:
# 파이썬 개념 정리
['?' for v in range(len([1,2,3,4]))]

['?', '?', '?', '?']

In [ ]:
def addGenre(name, flag = 0):
    # con 확인
    
    # 일치 키워드 관련 플래그 구문
    v = None

    if flag == 0:
        v = name

    elif flag == 1:
        v = '%' + name
    
    elif flag == 2:
        v = name + '%'
    
    else: 
        v = '%' + name + '%'

    
    cur.execute('SELECT pk FROM genre WHERE name=?', (v,))
    rst = cur.fetchone()

    if rst == None:
        cur.execute('INSERT INTO pk FROM genre(name) VALUES(?)', (name,))
        rst = cur.lastrowid
    else:
        rst = rst[0]
        
    return rst
    

In [108]:
# 알고리즘의 원리 : 일단 값이 없으면 무조건 넣는다.
p1 = '가수C'
p2 = '앨범5'

pk = addRow('artist', ['name'], [p1])
addRow('album', ['name', 'fk'], [p2, pk])

4

In [72]:
cur.execute('SELECT * FROM album')
cur.fetchall()

[(1, '앨범1', 1)]

In [110]:
pk1 = addRow('artist', ['name'], [p1])
pk2 = addRow('genre', ['name'], ['락'])
addRow('album', ['name', 'fk'], [p1,pk1,pk2])

False

In [101]:
con.commit()

In [74]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, '무제', 0, 0, 0, 1, 1), (2, '가수A', 0, 0, 0, 1, 1)]

In [75]:
# 내부적으로 번거롭고 귀찮기 떄문에 ORM이 나온다는 점을 알아채자.

In [96]:
# 수정 set => columns = value, 

def modRow(tablename, columns, values, newvalues):
    # 전제조건으로 column과 value의 개수를 맞추는 조건 주입.
    if len(columns) != len(values):
        return False
    
    # column 병합
    pk = addRow(tablename, columns, values)

    where = list()
    for i in range(len(columns)):
        where.append('='.join([columns[i], values[i]]))
    where = ' and '.join(where)

    s = list()
    for i in range(len(columns)):
        if newvalues[i]:
            s.append(columns[i] + '="' + newvalues[i] + '"') 
    s = ', '.join(s)


    print('UPDATE '+tablename+ ' SET ' + s + ' WHERE ' + where)
    cur.execute('UPDATE '+tablename+ 'SET ' + s + ' WHERE ' + where , (values))

    return cur.rowcount
    

In [ ]:
modRow('track', ['name', 'fk1', 'fk2'], ['가수A', 1, 2], ['노래1', None, None])

In [97]:
# 가수 기준 앨범
# 가수 1-* 앨범
cur.execute(''' 

    SELECT name1, COUNT(name2)
    FROM
    (SELECT artist.name AS name1, album.name AS name2 
    FROM artist
    LEFT JOIN album
    ON album.fk = artist.pk) AS A
    GROUP BY name1

''')
            
cur.fetchall()

[('가수A', 1), ('가수C', 0), ('가수D', 0), ('어쩌고', 0)]

In [98]:
# 가수 기준 앨범
# 가수 1-* 앨범
cur.execute(''' 

    SELECT DISTINCT(artist.name) 
    FROM artist
    LEFT JOIN album
    ON album.fk = artist.pk
    
''')
            
cur.fetchall()

[('가수A',), ('가수C',), ('가수D',), ('어쩌고',)]

In [ ]:
# DBMS의 연산을 이용하기 위해서 query를 잘 작성해야한다. 이번엔 함수 단위로 쿼리를 분기함. 함수로 쪼개야 한다. 
cur.execute(''' 

    SELECT D.name, B.name, C.name, A.name
    FROM track as A
    INNER JOIN album AS B
    ON A.fk = B.pk
    INNER JOIN genre AS C
    ON A.fk2 = C.pk
    INNER JOIN artist AS D
    ON D.pk = B.fk
    
''')
            
cur.fetchall()

In [100]:
#1. 장르별 노래 목록
cur.execute('''
    SELECT genre.pk, genre.name, track.pk, track.name
    FROM track
    INNER JOIN genre 
    ON track.fk2 = genre.pk
    ORDER BY genre.pk, track.pk ASC
''')
cur.fetchall()


[(1, '발라드', 1, '무제'), (1, '발라드', 2, '가수A'), (2, '락', 3, '가수A')]

In [ ]:
#2. 장르별 노래 갯수
cur.execute('''
    SELECT genre.pk, genre.name, COUNT(track.pk)
    FROM track
    INNER JOIN genre 
    ON track.fk2 = genre.pk
    ORDER BY genre.pk
    ORDER BY genre.pk, track.pk ASC
''')
cur.fetchall()


In [105]:
#3. 앨범별 노래 목록
cur.execute('''
    SELECT album.pk, album.name, track.pk, track.name
    FROM track
    INNER JOIN album 
    ON track.fk1 = album.pk
    ORDER BY album.fk, album.pk, track.pk ASC
''')
cur.fetchall()


[(1, '앨범1', 1, '무제'), (1, '앨범1', 2, '가수A'), (1, '앨범1', 3, '가수A')]

In [ ]:
#4. 앨범별 노래 갯수


In [ ]:
#5. 전체 플레이리스트에서 count > 5 초과이면서, 내림차순 -> 자주 듣는 목록.
cur.execute('''
    SELECT album.pk, album.name, genre.name, A.name, A.length, A.rating, A.count
    FROM track AS A
    INNER JOIN genre ON genre.pk = A.fk2
    INNER JOIN album ON album.pk = A.fk1
    INNER JOIN artist ON artist.pk = album.fk
    WHERE A.count > 5
    ORDER BY A.count > 5
    ORDER BY A.count DESC
''')
cur.fetchall()


In [ ]:

#6. 장르가 락일 때, 노래가 짧은 것 순서로.
cur.execute('''
    SELECT B.name, A.name
    FROM track AS A
    INNER JOIN genre AS B ON B.pk = A.fk2 AND B.name = '락'
    ORDER BY A.length ASC
''')
cur.fetchall()


In [ ]:

cur.execute('''
    SELECT B.name, A.name
    FROM track AS A
    INNER JOIN genre AS B ON B.pk = A.fk2
    WHERE genre.name = '락'
    ORDER BY A.length ASC
''')
cur.fetchall()


In [ ]:

cur.execute('''
    SELECT B.name, A.name
    FROM track AS A
    INNER JOIN genre AS B ON B.pk = A.fk2
    WHERE B.pk  = (SELECT pk FROM genre WHERE name = '락')
    ORDER BY A.length ASC
''')
cur.fetchall()